N-граммы - это языковые модели, которые используются для предсказания N-ного слова исходя из предыдущих N-1 слов.

P(w<sub>i</sub> | w<sub>i-N+1</sub>, w<sub>i-N+2</sub>, ..., w<sub>i-1</sub>) = C(w<sub>i-N+1</sub>, w<sub>i-N+2</sub>, ..., w<sub>i-1</sub>, w<sub>i</sub>) / C(w<sub>i-N+1</sub>, w<sub>i-N+2</sub>, ..., w<sub>i-1</sub>)  (1)

Словами: вероятность встретить слово w<sub>i</sub> после слов w<sub>i-N+1</sub>, w<sub>i-N+2</sub>, ..., w<sub>i-1</sub> (назовём это историей) равна отношению количества раз, которое всё сочетание слов вместе с w<sub>i</sub> втретилось в обучающей выборки, к количеству раз, которое в выборке встретилась история.

Пример: биграммы (N = 2)

In [ ]:
corpus = "John read her book. I read a different book. John read a book by Mulan."

Чтобы P(w<sub>i</sub> | w<sub>i-1</sub>) имело смысл для i = 1, добавим в начало каждого предложения специальный токен &lt;s&gt;. В общем случае таких токенов должно быть N - 1.

Чтобы вероятности всех последовательностей в сумме составляли 1, добавим в конец каждого предложения специальный токен &lt;/s&gt;

Теперь посчитаем вероятность P(John read a book):

P(John read a book) = P(John | &lt;s&gt;) * P(read | John) * P(a | read) * P(book | a) * P(&lt;/s&gt; | book)

А какова вероятность P(Mulan read a book)?

Сглаживание Лапласа: будем делать вид, что все возможные биграммы встретились на один раз больше, чем в реальности. Для этого в формуле (1) прибавим в числителе 1, а в знаменателе - размер словаря (включая &lt;s&gt; и &lt;/s&gt;):

P(w<sub>i</sub> | w<sub>i-N+1</sub>, w<sub>i-N+2</sub>, ..., 
w<sub>i-1</sub>) = (C(w<sub>i-N+1</sub>, w<sub>i-N+2</sub>, ..., w<sub>i-1</sub>, w<sub>i</sub>) + 1) / (C(w<sub>i-N+1</sub>, w<sub>i-N+2</sub>, ..., w<sub>i-1</sub>) + V) (2)

Обработка неизвестных слов: добавим в словарь токен &lt;unk&gt;, на который будем заменять все незнакомые слова (которые не входят в словарь). Благодаря сглаживанию Лапласа для всех N-грамм с этим словом будут ненулевые вероятности.

Задание: обучить две N-граммные модели на корпусе текстов студенческой конференции (две разные секции). Взять текст, не входящий в обучающую выборку, и попробовать определить его вероятность по каждой из двух моделей. Где вероятность больше, той секции, скорее всего, он и будет принадлежать.

1. Создать обучающие выборки

1.1. Отобрать тексты для каждой секции

1.2. Каждый текст разбить на предложения, очистить от пунктуации, привести к нижнему регистру

1.3. Каждое предложение разбить на слова, добавить &lt;s&gt; и &lt;/s&gt;, добавить в список предложений

1.4. Должно получиться два списка предложений (по одному на каждую секцию). Каждое предложение - это список слов, первое из которых - &lt;s&gt;, а последнее - &lt;/s&gt;. Следите, чтобы не было пустых предложений.

2. Обучить на каждой выборке N-граммную модель (N = 2 или 3)

2.1. Для каждой выборки составить лексикон (т.е. список встретившихся слов)

2.2. В лексикон должны входить &lt;s&gt;, &lt;/s&gt; и &lt;unk&gt;

2.3. На основе лексикона составить список всех теоретически возможных N-грамм

2.4. Для каждой N-граммы определить её вероятность по формуле (2)

2.5. Записать эти вероятности в словарь

2.6. Итого получилось два словаря, где ключи - N-граммы, значения - их вероятности. Эти словари и есть наши модели

3. Использовать модель для определения принадлежности текста

3.1. Взять текст, не входящий ни в одну из выборок (но желательно принадлежащий одной из двух секций)

3.2. Очистить и разбить на предложения по алгоритму из п. 1.

3.3. Для каждого предложения определить вероятность по каждой из двух моделей (не забывая заменять неизвестные модели слова на &lt;unk&gt;)

3.4. Сделать выводы

Модуль pickle для сериализации (хранения) данных

In [ ]:
import pickle

with open("conference_stud_2015.pkl", "rb") as f:
    data = pickle.load(f)

In [ ]:
with open("new_pickle.pkl", "wb") as f:
    pickle.dump(data, f)

Условный (!) пример обработки текста

In [ ]:
import re
text = data[2]["theses"]
text = re.sub("\s+", " ", text).strip()
text = text.lower()
text = text.split(".")
for sent in text:
    sent = sent.strip()
    if sent:
        sent = re.sub("[^a-zа-яё0-9 -]", "", sent)
        print(["<s>"] + sent.split() + ["</s>"])

['<s>', 'в', 'исследованиях', 'посвящённых', 'анализу', 'современного', 'состояния', 'русского', 'языка', 'в', 'качестве', 'одной', 'из', 'главных', 'тенденций', 'выделяется', 'тенденция', 'к', 'аналитизму', 'которая', 'проявляет', 'себя', 'в', 'увеличении', 'числа', 'слов', 'выражающих', 'грамматическое', 'значение', 'вне', 'словоформы', '</s>']
['<s>', 'среди', 'явлений', 'выступающих', 'показателями', 'обозначенного', 'направления', 'учёные', 'особо', 'выделяют', 'пополнение', 'группы', 'аналитических', 'прилагательных', 'или', 'аналитов', 'единиц', 'агглютинативного', 'характера', 'являющихся', 'модификатором', 'производящей', 'базы', 'в', 'акте', 'словообразования', '</s>']
['<s>', 'в', 'результате', 'увеличения', 'группы', 'аналитов', 'закономерным', 'становится', 'появление', 'в', 'языке', 'новых', 'словообразовательных', 'моделей', 'и', 'типов', '</s>']
['<s>', 'в', 'центре', 'нашего', 'исследования', 'рассмотрение', 'одного', 'из', 'продуктивных', 'типов', 'в', 'котором', 'фор

Отфильтруем данные только одной секции

In [ ]:
phonetics_data = [i for i in data if i["section"] == "Фонетика"]

phonetics_corpus = []
for entry in phonetics_data:
    text = entry["theses"]
    # разбили на предложения
    # убрали знаки препинания
    # привели к нижнему регистру
    # предложения разбили на слова
    # добавили теги
    sentences = []
    phonetics_corpus += sentences

Посмотрим на список секций

In [ ]:
set([i["section"] for i in data])

{'Балканистика. Византинистика. Неоэллинистика',
 'Библеистика',
 'Будетляне. Гипотеза в филологии: TED TALKS',
 'Грамматика (романо-германская филология)',
 'Грамматика и семантика русского языка',
 'История зарубежных литератур',
 'История русского языка',
 'Кино|Текст',
 'Классическая филология',
 'Лексикология (романо-германская филология)',
 'Лексикология и стилистика русского языка',
 'Лингвометодические основы описания и изучения русского языка как иностранного',
 'Народная культура в древнем и новом слове',
 'Общее языкознание',
 'Переводоведение (романо-германская филология)',
 'Пленарное заседание',
 'Прикладная и математическая лингвистика',
 'Психолингвистика',
 'Русская литература: история',
 'Русская литература: теория, поэтика',
 'Славяно-германская компаративистика',
 'Славянская филология: литературоведение',
 'Славянская филология: языкознание',
 'Финно-угорская филология',
 'Фольклор и мифология',
 'Фонетика'}

Функция enumerate()

In [ ]:
a = ["a", "b", "c", "d"]

for i, el in enumerate(a):
    print(i, el)
    a[i] = el * 2

print(a)

0 a
1 b
2 c
3 d
['aa', 'bb', 'cc', 'dd']


Функция itertools.product() для генерации N-грамм

In [ ]:
from itertools import product

words = ["a", "b", "c"]
ngram_order = 3

word_comb = product(words, repeat=ngram_order)
print(list(word_comb))

[('a', 'a', 'a'), ('a', 'a', 'b'), ('a', 'a', 'c'), ('a', 'b', 'a'), ('a', 'b', 'b'), ('a', 'b', 'c'), ('a', 'c', 'a'), ('a', 'c', 'b'), ('a', 'c', 'c'), ('b', 'a', 'a'), ('b', 'a', 'b'), ('b', 'a', 'c'), ('b', 'b', 'a'), ('b', 'b', 'b'), ('b', 'b', 'c'), ('b', 'c', 'a'), ('b', 'c', 'b'), ('b', 'c', 'c'), ('c', 'a', 'a'), ('c', 'a', 'b'), ('c', 'a', 'c'), ('c', 'b', 'a'), ('c', 'b', 'b'), ('c', 'b', 'c'), ('c', 'c', 'a'), ('c', 'c', 'b'), ('c', 'c', 'c')]


Обработка каждой N-граммы в предложении: сделаем списки N-грамм и N-1-грамм.

In [ ]:
ngram_order = 3

text = [["a", "b", "c", "d", "e"], ["a", "f", "d", "g", "n"], ["e", "q", "y", "e"]]

ngram_list = []
ngram_part_list = []

for sent in text:
    sent = ["<s>"] * (ngram_order - 1) + sent + ["</s>"]
    for i in range(len(sent) - ngram_order + 1):
        ngram = sent[i:i + ngram_order]
        ngram_list.append(tuple(ngram))
    for i in range(len(sent) - ngram_order + 2):
        ngram_part = sent[i:i + ngram_order - 1]
        ngram_part_list.append(tuple(ngram_part))

print(ngram_list)
print(ngram_part_list)

[('<s>', '<s>', 'a'), ('<s>', 'a', 'b'), ('a', 'b', 'c'), ('b', 'c', 'd'), ('c', 'd', 'e'), ('d', 'e', '</s>'), ('<s>', '<s>', 'a'), ('<s>', 'a', 'f'), ('a', 'f', 'd'), ('f', 'd', 'g'), ('d', 'g', 'n'), ('g', 'n', '</s>'), ('<s>', '<s>', 'e'), ('<s>', 'e', 'q'), ('e', 'q', 'y'), ('q', 'y', 'e'), ('y', 'e', '</s>')]
[('<s>', '<s>'), ('<s>', 'a'), ('a', 'b'), ('b', 'c'), ('c', 'd'), ('d', 'e'), ('e', '</s>'), ('<s>', '<s>'), ('<s>', 'a'), ('a', 'f'), ('f', 'd'), ('d', 'g'), ('g', 'n'), ('n', '</s>'), ('<s>', '<s>'), ('<s>', 'e'), ('e', 'q'), ('q', 'y'), ('y', 'e'), ('e', '</s>')]


Посчитаем, сколько раз они встретились:

In [ ]:
from collections import Counter
ngram_counter = Counter(ngram_list)
ngram_part_counter = Counter(ngram_part_list)

Определим размер лексикона:

In [ ]:
lexicon = sorted({w for sent in text for w in sent}) + ["<s>", "</s>", "<unk>"]
V = len(lexicon)

Посчитаем вероятность тестовой N-граммы по формуле (2):

In [ ]:
test_ngram = ("<s>", "<s>", "a")
hist = tuple(test_ngram[:-1])
p = (ngram_counter.get(test_ngram, 0) + 1) / (ngram_part_counter.get(hist, 0) + V)
print(p)

0.1875
